In [13]:
import pandas as pd
import numpy as np 
import plotly.express as px


In [4]:
df=pd.read_csv("df_CostRepor.csv")

In [6]:
print(df.columns.tolist())

['ACCT', 'BUDGET', 'COST TO DATE', 'REF', 'TIPO', 'Etapa', 'VARIANCE']


In [50]:
df.replace("nan", np.nan, inplace=True)


for col in ["COST TO DATE", "BUDGET", "VARIANCE"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')  

total_cost = df["COST TO DATE"].sum(skipna=True)
total_budget = df["BUDGET"].sum(skipna=True)

total_variance = df["VARIANCE"].sum(skipna=True)

cpi = round(total_cost / total_budget, 2) if total_budget != 0 else 0

print("RESUMEN GENERAL")
print(f"→ Costo Total a la Fecha: ${total_cost:,.2f}")
print(f"→ Presupuesto Total: ${total_budget:,.2f}")
print(f"→ Variación Total: ${total_variance:,.2f}")

print(f"→ Índice de Cumplimiento Presupuestal (CPI): {cpi}") 
# mide la eficiencia del uso del presupuesto hasta la fecha. Te dice si estás gastando más, menos o justo lo que deberías.

beneficiadas = df.sort_values("VARIANCE", ascending=False).head(5)
print("\n Cuentas Más Beneficiadas:")
print(beneficiadas[["ACCT", "COST TO DATE", "BUDGET", "VARIANCE"]].dropna().to_string(index=False))

impactadas = df.sort_values("VARIANCE", ascending=True).head(5)

print("\n Cuentas Más Impactadas:")
print(impactadas[["ACCT", "COST TO DATE", "BUDGET", "VARIANCE"]].dropna().to_string(index=False))

RESUMEN GENERAL
→ Costo Total a la Fecha: $403,115,948.29
→ Presupuesto Total: $405,293,733.00
→ Variación Total: $3,000,835.78
→ Índice de Cumplimiento Presupuestal (CPI): 0.99

 Cuentas Más Beneficiadas:
    ACCT  COST TO DATE      BUDGET    VARIANCE
3412-127      2155.170  2701500.00 2699344.830
    3412   4270057.826  6880105.37 2610047.544
    3400  12724039.845 15001515.11 2277475.265
7003-004  10574177.830 11643573.80 1069395.970
    7003  10574177.830 11643573.80 1069395.970

 Cuentas Más Impactadas:
    ACCT  COST TO DATE      BUDGET     VARIANCE
    1400  13899922.025 11960556.35 -1939365.675
    1401   8807210.200  7612289.00 -1194921.200
6137-003         0.000  -849515.52  -849515.520
    6137         0.000  -849515.52  -849515.520
1499-001   1649770.708   818091.03  -831679.678


In [65]:
tipo_cost = df.groupby('TIPO')['COST TO DATE'].sum().reset_index()

fig = px.pie(tipo_cost, values='COST TO DATE', names='TIPO',title='Distribución del Costo Total por TIPO')  

fig.update_layout(title={'text': "<b> Distribución del Costo Total por TIPO</b>", 'x':0.5}, title_font_size=22)

fig.show()

Esta gráfica de pastel muestra cómo se distribuye el costo total entre los diferentes tipos de gasto. Cada segmento representa un tipo, y el tamaño del segmento indica qué porcentaje del gasto total corresponde a ese tipo.

Esta visualización permite detectar con facilidad qué áreas dominan el gasto operativo y abre la puerta a preguntas estratégicas


In [32]:
ahorros = df.sort_values("VARIANCE", ascending=False).head(10)

fig= px.bar(top_savings, x="VARIANCE", y="ACCT",labels={"VARIANCE": "VARIANCE ($)", "ACCT": "Cuenta"},color="VARIANCE", color_continuous_scale="greens", height=500)

fig.update_layout(title={'text': "<b> Top 10 Cuentas con Mayor Ahorro</b>", 'x':0.5},title_font_size=22)

fig.show()


Se muestran las 10 cuentas con mayor ahorro con respecto al presupuesto asignado. El objetivo principal es visualizar cuáles han tenido una ejecución presupuestal más baja, lo que puede indicar una mala gestión de recursos, un subejercicio o incluso una sobreestimación del presupuesto original.

In [49]:
df["CUENTA_GLOBAL"] = df["ACCT"].astype(str).str.extract(r"^(\d{4})")

df_pie = df.groupby("CUENTA_GLOBAL", as_index=False)["COST TO DATE"].sum().sort_values("COST TO DATE", ascending=False).head(35)

fig = px.pie(df_pie, names="CUENTA_GLOBAL", values="COST TO DATE",title="<b>Top 10 Cuentas Globales con Mayor Costo</b>")

fig.update_layout(title={'x':0.5}, title_font_size=22)
fig.show()

Esta gráfica de pastel facilita la visualización y comparación del uso del presupuesto entre cuentas globales. Al ser interactiva, permite identificar qué cuentas representan los mayores porcentajes del gasto total.

Se usuario puede seleccionar cuentas específicas para analizar su peso relativo dentro del total, lo que resulta útil para detectar concentraciones de recursos, lo que también ayuda a tomar decisiones más informadas sobre la distribución presupuestal y posibles ajustes financieros.

En resumen, permite no solo observar cuánto se gasta por cuenta, sino también qué tan relevante es ese gasto respecto al total general.

In [61]:
df["EFICIENCIA"] = np.where(df["VARIANCE"] >= 0, "Ahorro", "Sobrecosto")

df_bubble = df.dropna(subset=["BUDGET", "COST TO DATE", "VARIANCE"]).copy()

df_bubble["VARIANCE"] = df_bubble["VARIANCE"].abs() #se toman los valores abs para evitar errores

fig = px.scatter(df_bubble, 
                 x="BUDGET", 
                 y="COST TO DATE", 
                 size="VARIANCE", 
                 color="EFICIENCIA",
                 hover_name="ACCT",
                 color_discrete_map={"Ahorro": "green", "Sobrecosto": "red"},
                 title="<b>Impacto Financiero por Cuenta (Presupuesto vs Costo Real)</b>",
                 labels={"BUDGET": "BUDGET ($)", "COST TO DATE": "COST TO DATE ($)"},
                 size_max=60,height=600)

fig.update_layout(title={'x': 0.5}, title_font_size=24)
fig.show()

Esta gráfica tipo scatterplot permite analizar visualmente la relación entre el presupuesto asignado y el gasto real (Cost to Date) por cuenta. Cada punto representa una cuenta específica, donde el eje X indica el presupuesto y el eje Y muestra el costo real. El tamaño de cada burbuja refleja la magnitud de la variación entre ambos valores, mientras que el color permite identificar rápidamente el tipo de resultado: verde para cuentas con ahorro y rojo para cuentas con sobrecosto. Además, al ser interactiva, permite consultar los detalles de cada cuenta con solo pasar el cursor sobre las burbujas, haciendo de esta herramienta una forma intuitiva y poderosa de explorar la información financiera.